# Preprocess the data

## Rigid registration and skull extraction

In [1]:
import os
from os.path import expanduser as eu
from headctools.preprocessing import ct_skull, utils

In [2]:
imgs = eu('~/Code/datasets/cq500/converted/selected/test')
fixed_img = os.path.join(imgs, 'CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz')
# Registered images will be saved in a subfolder with the name of the fixed img

params = {'reg_atlas_path': fixed_img,
          'overwrite_images': False,
          # 'skip_registration': True  # Non-default params
          }

ct_skull(imgs, params = params)  # Preprocess the imgs

Getting nii.gz images from /home/fmatzkin/Code/datasets/cq500/converted/selected/test
Found 2 images
 No save path specified. Using default save path.
Preprocessing {'image': '/home/fmatzkin/Code/datasets/cq500/converted/selected/test/CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz'}
  File /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz already exists..
Preprocessing {'image': '/home/fmatzkin/Code/datasets/cq500/converted/selected/test/CQ500-CT-405_CT Thin Plain.nii.gz'}
  File /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/CQ500-CT-405_CT Thin Plain.nii.gz already exists..
  Saved images in /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull


'/home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull'

## Mesh atlas construction

### Decimate meshes

In [3]:
meshes_folder = '~/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull'
decimate_factor = 0.01
decimated_meshes = utils.decimate_meshes(meshes_folder, decimate_factor,
                                         overwrite=False)

Decimating meshes...
  Input folder: /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull
  Found 2 meshes
    [1/2] Input mesh: CQ500-CT-312_CT PRE CONTRAST THIN.stl
    Mesh already exists, skipping...
    [2/2] Input mesh: CQ500-CT-405_CT Thin Plain.stl
    Mesh already exists, skipping...


### Deformetrica
#### Atlas estimation

In [4]:
from src.deformetrica import estimate_atlas, estimate_registration
template_path, dataset_paths = decimated_meshes[0], decimated_meshes[1:]

In [4]:
estimate_atlas(template_path, dataset_paths)

Logger has been set to: INFO
>> No initial CP spacing given: using diffeo kernel width of 40.0
OMP_NUM_THREADS was not found in environment variables. An automatic value will be set.
OMP_NUM_THREADS will be set to 2
>> No specified state-file. By default, Deformetrica state will by saved in file: output/deformetrica-state.p.
>> Set of 80 control points defined.
>> Momenta initialized to zero, for 4 subjects.
>> Started estimator: GradientAscent


/home/franco/anaconda3/envs/msr/lib/python3.8/site-packages/deformetrica/core/observations/deformable_objects/landmarks/surface_mesh.py:37: UserWarning:

This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)



------------------------------------- Iteration: 0 -------------------------------------
>> Log-likelihood = -4.609E+05 	 [ attachment = -4.609E+05 ; regularity = 0.000E+00 ]
>> Step size and gradient norm: 
		3.349E-06   and   2.986E+05 	[ landmark_points ]
		6.220E-05   and   1.608E+04 	[ momenta ]
------------------------------------- Iteration: 1 -------------------------------------
>> Log-likelihood = -4.439E+05 	 [ attachment = -4.439E+05 ; regularity = -3.107E+00 ]
>> Step size and gradient norm: 
		5.024E-06   and   2.927E+05 	[ landmark_points ]
		9.330E-05   and   1.577E+04 	[ momenta ]
------------------------------------- Iteration: 2 -------------------------------------
>> Log-likelihood = -4.195E+05 	 [ attachment = -4.195E+05 ; regularity = -1.895E+01 ]
>> Step size and gradient norm: 
		7.536E-06   and   2.833E+05 	[ landmark_points ]
		1.400E-04   and   1.527E+04 	[ momenta ]
------------------------------------- Iteration: 3 -------------------------------------
>> 

In [6]:
estimate_registration(template_path, dataset_paths)

Logger has been set to: INFO
>> No initial CP spacing given: using diffeo kernel width of 40.0
OMP_NUM_THREADS was not found in environment variables. An automatic value will be set.
OMP_NUM_THREADS will be set to 4
>> No specified state-file. By default, Deformetrica state will by saved in file: /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/reg_CQ500-CT-312_CT PRE CONTRAST THIN_decimated_1perc/deformetrica-state.p.
>> Set of 80 control points defined.
>> Momenta initialized to zero, for 1 subjects.
>> Started estimator: GradientAscent


/home/fmatzkin/Code/envs/anaconda3/envs/msr/lib/python3.8/site-packages/deformetrica/core/observations/deformable_objects/landmarks/surface_mesh.py:37: UserWarning:

This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)



------------------------------------- Iteration: 0 -------------------------------------
>> Log-likelihood = -5.269E+04 	 [ attachment = -5.269E+04 ; regularity = 0.000E+00 ]
>> Step size and gradient norm: 
		1.955E-04   and   5.114E+03 	[ momenta ]
------------------------------------- Iteration: 1 -------------------------------------
>> Log-likelihood = -4.777E+04 	 [ attachment = -4.776E+04 ; regularity = -3.230E+00 ]
>> Step size and gradient norm: 
		2.933E-04   and   4.730E+03 	[ momenta ]
------------------------------------- Iteration: 2 -------------------------------------
>> Log-likelihood = -4.159E+04 	 [ attachment = -4.157E+04 ; regularity = -1.836E+01 ]
>> Step size and gradient norm: 
		4.400E-04   and   4.175E+03 	[ momenta ]
------------------------------------- Iteration: 3 -------------------------------------
>> Log-likelihood = -3.462E+04 	 [ attachment = -3.456E+04 ; regularity = -5.715E+01 ]
>> Step size and gradient norm: 
		6.599E-04   and   3.429E+03 	[ mom

## Image with defect registration test

Test-time mesh registration simulation:

1.  Grab an image without defect.
2.  Mesh and register the template to the skull.
3.  Simulate a craniectomy.
4.  Mesh the skull with the defect.
5.  Register (2) to (4).

### Simulate defect

In [5]:
from headctools.tools.offline_augmentation import simulate_and_save

input_folder = os.path.expanduser(meshes_folder)
files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('_image.nii.gz')]

for fpath in files:
    simulate_and_save(fpath, 1, os.path.join(input_folder, 'cran'))

KeyboardInterrupt: 

### Convert to stl

In [ ]:
!python ~/Code/some-gists/mesh/nii_to_stl_marching_cubes.py /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/

### Decimate outputs

In [6]:
meshes_folder = '/home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/'
decimate_factor = 0.01
decimated_meshes_def = utils.decimate_meshes(meshes_folder, decimate_factor,
                                             overwrite=False)

Decimating meshes...
  Input folder: /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/
  Found 2 meshes
    [1/2] Input mesh: CQ500-CT-312_CT PRE CONTRAST THIN_0_sim.stl
    Mesh already exists, skipping...
    [2/2] Input mesh: CQ500-CT-405_CT Thin Plain_0_sim.stl
    Mesh already exists, skipping...


### Register atlas to simulated defect images
Here the template image (the subject 312 with the full skull) will be registered to every skull with a defect.
For each patient with a defect, a folder will be created with two outputs:
 1) DeterministicAtlas__EstimatedParameters__Template_skull.vtk
 2) DeterministicAtlas__Reconstruction__skull__IMAGE_NAME.vtk

(1) Aligns better with the postoperative image if it's the same subject image, otherwise (2) will be the best adaptation of the template to this image.

In [17]:
defect_imgs = '/home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/'
cran_paths = [os.path.join(defect_imgs, f) for f in os.listdir(defect_imgs)
              if f.endswith('.vtk')]
template_path = decimated_meshes[0]  # Preprocessed + dmtd complete skull (312)

for fixed_path in cran_paths:
    out_path = os.path.join(defect_imgs, 'reg_' +
                            os.path.splitext(os.path.split(fixed_path)[1])[0]
                            )
    estimate_registration(template_path, [fixed_path], out_path)

Logger has been set to: INFO
>> No initial CP spacing given: using diffeo kernel width of 40.0
OMP_NUM_THREADS was not found in environment variables. An automatic value will be set.
OMP_NUM_THREADS will be set to 4
context has already been set
>> No specified state-file. By default, Deformetrica state will by saved in file: /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/reg_CQ500-CT-312_CT PRE CONTRAST THIN_0_sim_decimated_1perc/deformetrica-state.p.
>> Set of 80 control points defined.
>> Momenta initialized to zero, for 1 subjects.
>> Started estimator: GradientAscent
------------------------------------- Iteration: 0 -------------------------------------
>> Log-likelihood = -6.289E+03 	 [ attachment = -6.289E+03 ; regularity = 0.000E+00 ]
>> Step size and gradient norm: 
		3.664E-03   and   2.730E+02 	[ momenta ]
------------------------------------- Iteration: 1 -------------------------------------
>> Log-likelihood =

### Convert to stl
Needed for pymesh

In [20]:
import vtk

def vtk_to_stl(filepath, outfile):
    if os.path.isfile(filepath):
        print(f"Creating {outfile}")
        reader = vtk.vtkGenericDataObjectReader()
        reader.SetFileName(filepath)
        reader.Update()
        writer = vtk.vtkSTLWriter()
        writer.SetInputConnection(reader.GetOutputPort())
        writer.SetFileName(outfile)
        return writer.Write()==1
    return False

files = [
    '/home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/reg_CQ500-CT-312_CT PRE CONTRAST THIN_0_sim_decimated_1perc/DeterministicAtlas__Reconstruction__skull__subject_CQ500-CT-312_CT PRE CONTRAST THIN_0_sim_decimated_1perc.vtk',
    '/home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/reg_CQ500-CT-405_CT Thin Plain_0_sim_decimated_1perc/DeterministicAtlas__Reconstruction__skull__subject_CQ500-CT-405_CT Thin Plain_0_sim_decimated_1perc.vtk',
    '/home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/CQ500-CT-312_CT PRE CONTRAST THIN_0_sim_decimated_1perc.vtk',
    '/home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/CQ500-CT-405_CT Thin Plain_0_sim_decimated_1perc.vtk'
]

stl_registered = [f.replace('.vtk', '.stl') for f in files]

for file in files:
    out_dir = file.replace('.vtk', '.stl')
    vtk_to_stl(file, out_dir)

Creating /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/reg_CQ500-CT-312_CT PRE CONTRAST THIN_0_sim_decimated_1perc/DeterministicAtlas__Reconstruction__skull__subject_CQ500-CT-312_CT PRE CONTRAST THIN_0_sim_decimated_1perc.stl
Creating /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/reg_CQ500-CT-405_CT Thin Plain_0_sim_decimated_1perc/DeterministicAtlas__Reconstruction__skull__subject_CQ500-CT-405_CT Thin Plain_0_sim_decimated_1perc.stl
Creating /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/CQ500-CT-312_CT PRE CONTRAST THIN_0_sim_decimated_1perc.stl
Creating /home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/CQ500-CT-405_CT Thin Plain_0_sim_decimated_1perc.stl


### Difference
Difference of the transformed template mesh with the craniectomy mesh

In [24]:
import trimesh
cran = trimesh.load_mesh(craniect_imgs[0])

craniect_imgs = ['/home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/CQ500-CT-312_CT PRE CONTRAST THIN_0_sim_decimated_1perc.stl',
                 '/home/fmatzkin/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/def/meshes/decimated/CQ500-CT-405_CT Thin Plain_0_sim_decimated_1perc.stl']

cran = stl.Stl(craniect_imgs[0])  # craniect orig mesh
tempreg = stl.Stl(files[0])  # registered template mesh

output_mesh = pymesh.boolean(tempreg, cran, operation="difference", engine="igl")
pymesh.save_mesh("312-diff.stl", output_mesh)